<a href="https://colab.research.google.com/github/couyang24/general_learning-tiffany/blob/master/colab_titanic_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Need to mount Drive on or upload kaggle.json

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir ~/.kaggle/

rm: cannot remove '/root/.kaggle': No such file or directory


In [3]:
!cp drive/My\ Drive/input/kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.9MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 29.4MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.26MB/s]


In [5]:
# Load Package
# import numpy as np
import pandas as pd
import seaborn as sns
import featuretools

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
import featuretools as ft

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    LabelEncoder,
    OrdinalEncoder,
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [8]:
# Load data
train_df= pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [9]:
# Save data
target = train_df[["Survived"]]
submission = test_df[["PassengerId"]]

In [10]:
# Join and Clean
combine = pd.concat([train_df, test_df])

In [11]:
# EDA
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [12]:
combine.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [13]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

In [14]:
combine['Title'] = combine.Name.apply(lambda x: x.split(".")[0].split(",")[1].strip()).replace(mapping)

In [15]:
combine.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

In [16]:
# combine['Sex2'] = combine['Sex'].apply(lambda x: 0 if x=='female' else 1)

In [17]:
# class ModifiedLabelEncoder(LabelEncoder):

#     def fit_transform(self, y, *args, **kwargs):
#         return super().fit_transform(y)

#     def transform(self, y, *args, **kwargs):
#         return super().transform(y)

In [18]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encode", OrdinalEncoder()),
])

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

In [19]:
combine[['Sex','Embarked', 'Title']] = categorical_transformer.fit_transform(combine[['Sex', 'Embarked', 'Title']])

In [20]:
combine[['Age', 'Fare']] = numeric_transformer.fit_transform(combine[['Age', 'Fare']])

In [21]:
es = ft.EntitySet(id = 'titanic_data')

es = es.entity_from_dataframe(entity_id = 'combine', dataframe = combine.drop(['Survived'], axis=1), 
                              variable_types = 
                              {
                                  'Embarked': ft.variable_types.Categorical,
                                  'Sex': ft.variable_types.Boolean,
                                  'Title': ft.variable_types.Categorical
                              },
                              index = 'PassengerId')

es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
  Relationships:
    No relationships

In [22]:
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Embarked', index='Embarked')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Sex', index='Sex')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Title', index='Title')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Pclass', index='Pclass')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Parch', index='Parch')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='SibSp', index='SibSp')
es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
    Embarked [Rows: 3, Columns: 1]
    Sex [Rows: 2, Columns: 1]
    Title [Rows: 7, Columns: 1]
    Pclass [Rows: 3, Columns: 1]
    Parch [Rows: 8, Columns: 1]
    SibSp [Rows: 7, Columns: 1]
  Relationships:
    combine.Embarked -> Embarked.Embarked
    combine.Sex -> Sex.Sex
    combine.Title -> Title.Title
    combine.Pclass -> Pclass.Pclass
    combine.Parch -> Parch.Parch
    combine.SibSp -> SibSp.SibSp

In [23]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(primitives[primitives['type'] == 'aggregation'].shape[0])

,name,type,description
0,count,aggregation,Counts the number of non null values.
1,mode,aggregation,Finds the most common element in a categorical feature.
2,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
3,sum,aggregation,Sums elements of a numeric or boolean feature.
4,all,aggregation,Test if all values are 'True'.
5,percent_true,aggregation,Finds the percent of 'True' values in a boolean feature.
6,time_since_last,aggregation,Time since last related instance.
7,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
8,any,aggregation,Test if any value is 'True'.
9,num_true,aggregation,Finds the number of 'True' values in a boolean.


In [24]:
primitives[primitives['type'] == 'transform'].head(primitives[primitives['type'] == 'transform'].shape[0])

,name,type,description
19,month,transform,Transform a Datetime feature into the month.
20,year,transform,Transform a Datetime feature into the year.
21,and,transform,"For two boolean values, determine if both values are 'True'."
22,longitude,transform,Returns the second value on the tuple base feature.
23,week,transform,Transform a Datetime feature into the week.
24,minute,transform,Transform a Datetime feature into the minute.
25,percentile,transform,"For each value of the base feature, determines the percentile in relation"
26,diff,transform,Compute the difference between the value of a base feature and the previous value.
27,second,transform,Transform a Datetime feature into the second.
28,days_since,transform,"For each value of the base feature, compute the number of days between it"


In [25]:
features, feature_names = ft.dfs(entityset = es, 
                                 target_entity = 'combine', 
#                                  trans_primitives=['subtract_numeric', 'add_numeric', 'divide_numeric', 'multiply_numeric'],
                                 max_depth = 2)

In [26]:
feature_names

[<Feature: Pclass>,
 <Feature: Age>,
 <Feature: SibSp>,
 <Feature: Parch>,
 <Feature: Fare>,
 <Feature: Embarked>,
 <Feature: Sex>,
 <Feature: Title>,
 <Feature: Embarked.SUM(combine.Age)>,
 <Feature: Embarked.SUM(combine.Fare)>,
 <Feature: Embarked.STD(combine.Age)>,
 <Feature: Embarked.STD(combine.Fare)>,
 <Feature: Embarked.MAX(combine.Age)>,
 <Feature: Embarked.MAX(combine.Fare)>,
 <Feature: Embarked.SKEW(combine.Age)>,
 <Feature: Embarked.SKEW(combine.Fare)>,
 <Feature: Embarked.MIN(combine.Age)>,
 <Feature: Embarked.MIN(combine.Fare)>,
 <Feature: Embarked.MEAN(combine.Age)>,
 <Feature: Embarked.MEAN(combine.Fare)>,
 <Feature: Embarked.COUNT(combine)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Pclass)>,
 <Feature: Embarked.NUM_UNIQUE(combine.SibSp)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Parch)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Sex)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Title)>,
 <Feature: Embarked.MODE(combine.Pclass)>,
 <Feature: Embarked.MODE(combine.SibSp)>,
 <Feature:

In [27]:
len(feature_names)

146

In [28]:
features.isnull().sum()

Pclass                          0
Age                             0
SibSp                           0
Parch                           0
Fare                            0
                               ..
SibSp.MODE(combine.Pclass)      0
SibSp.MODE(combine.Parch)       0
SibSp.MODE(combine.Embarked)    0
SibSp.MODE(combine.Sex)         0
SibSp.MODE(combine.Title)       0
Length: 146, dtype: int64

In [29]:
class RemoveLowInfo(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        keep = [column for column in df.columns if df[column].value_counts(normalize=True).reset_index(drop=True)[0]<self.threshold]
        return df[keep]

In [32]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

In [33]:
impute_median = FunctionTransformer(lambda x: x.fillna(x.median()), validate=False)

In [34]:
normalize = FunctionTransformer(lambda x: (x-x.mean())/x.std(), validate=False)

In [35]:
from sklearn.decomposition import PCA

In [36]:
transformer = Pipeline([
    ("imputer", impute_median),
    ('removelowinfo',RemoveLowInfo(threshold=.95)),
    ('scaler', normalize),
])

In [37]:
clean_features = transformer.fit_transform(features)

In [41]:
!pip install catboost

     |████████████████████████████████| 64.8MB 62kB/s 


In [42]:
from sklearn.linear_model  import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
import xgboost as xgb
import lightgbm as lgb
import catboost as cgb

In [43]:
methods = [
           ('logistic', LogisticRegression(solver='lbfgs')), 
#            ('sgd', SGDClassifier()), 
           ('tree', DecisionTreeClassifier()),
           ('bag', BaggingClassifier()),
           ('xgb', xgb.XGBClassifier(max_depth=3)),
           ('lgb', lgb.LGBMClassifier(max_depth=3)),
#            ('cgb', cgb.CatBoostClassifier(max_depth=3,silent=True)),
           ('ada', AdaBoostClassifier()),
           ('gbm', GradientBoostingClassifier()),
           ('rf', RandomForestClassifier(n_estimators=100)),
#            ('svc', LinearSVC()),
#            ('rbf', SVC()),
           ('nb', Pipeline([('pca', PCA()), ('gnb', GaussianNB())])),
           ('nn', MLPClassifier()),
           ('knn', KNeighborsClassifier()),
          ]


ensemble = VotingClassifier(
        methods,
        voting='soft', 
#         weights=[1,1,1,1,2,2,1,1],
#         flatten_transform=True,
)

clf = Pipeline([
#          ('transformer', transformer),
     ('ensemble', ensemble),
])

clf.fit(clean_features.iloc[:train_df.shape[0],:], target)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

Pipeline(memory=None,
         steps=[('ensemble',
                 VotingClassifier(estimators=[('logistic',
                                               LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                                                                  random_st

In [44]:
submission['Survived'] = pd.DataFrame(clf.predict(clean_features.iloc[train_df.shape[0]:,:]))

In [45]:
print(submission.dtypes)

PassengerId    int64
Survived       int64
dtype: object


In [50]:
submission.to_csv("titanic_submission.csv", index=False)